In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Get the path to the dataset folder.
dataset_dir = "C:/Users/HP 840 G1/Documents/VS Code Projects/Workspace Learning/DataScience/Jupyter Notebook/Thesis_CSE/datasets/Rice_Leaf_Disease_Images_Dataset-4"

# Get the list of diseases.
diseases = os.listdir(dataset_dir)

# Create the lists of training and validation images and labels.
train_images = []
train_labels = []
val_images = []
val_labels = []

# Iterate over the diseases.
for disease in diseases:
    # Get the path to the folder of the disease.
    disease_dir = os.path.join(dataset_dir, disease)

    # Iterate over the images in the folder.
    for image_name in os.listdir(disease_dir):
        # Get the path to the image.
        image_path = os.path.join(disease_dir, image_name)

        # Read the image.
        image = cv2.imread(image_path)

        # Randomly assign the image to the training or validation set.
        split = np.random.rand()
        if split < 0.8:
            train_images.append(image)
            train_labels.append(disease)
        else:
            val_images.append(image)
            val_labels.append(disease)

# Convert the lists of images and labels to NumPy arrays.
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

datasets = (train_images, train_labels), (val_images, val_labels)

print(train_labels)

['RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBacterialBlight' 'RiceBacterialBlight' 'RiceBacterialBlight'
 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast'
 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast'
 'RiceBlast' 'RiceBlast' 'RiceBlast' 'RiceBlast' '

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def faster_rcnn(image_size, num_classes, num_landmarks):
    """
    Creates a Faster R-CNN model with ResNet50 backbone and landmark regression layer.

    Args:
      image_size: The size of the input images.
      num_classes: The number of object classes.
      num_landmarks: The number of landmarks.

    Returns:
      A Keras model.
    """

    # Create the ResNet50 model.
    resnet50 = ResNet50(include_top=False, weights='imagenet')

    # Create the RPN.
    rpn_conv2d = Conv2D(256, (3, 3), padding='same')(resnet50.output)
    rpn_output = tf.keras.layers.Dense(4)(rpn_conv2d)

    # Create the Fast R-CNN network.
    fast_rcnn_conv2d = Conv2D(128, (3, 3), padding='same')(rpn_conv2d)
    fast_rcnn_output = tf.keras.layers.Dense(num_classes)(fast_rcnn_conv2d)

    # Create the landmark regression layer.
    landmark_regression = Conv2D(
        2 * num_landmarks, (3, 3), padding='same')(fast_rcnn_conv2d)

    # Create the model.
    model = Model(inputs=resnet50.input, outputs=[
                  rpn_output, fast_rcnn_output, landmark_regression])

    return model

def draw_bounding_box(image, bounding_box):
    """
    Draws a bounding box on the image.

    Args:
      image: The input image.
      bounding_box: Bounding box coordinates (x1, y1, x2, y2).

    Returns:
      An image with the bounding box drawn.
    """
    x1, y1, x2, y2 = bounding_box
    image_with_box = image.copy()
    cv2.rectangle(image_with_box, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image_with_box

def draw_landmarks(image, landmarks):
    """
    Draws landmarks on the image.

    Args:
      image: The input image.
      landmarks: Landmark coordinates.

    Returns:
      An image with landmarks drawn.
    """
    image_with_landmarks = image.copy()
    for landmark in landmarks:
        x, y = landmark
        cv2.circle(image_with_landmarks, (x, y), 5, (255, 0, 0), -1)
    return image_with_landmarks


def main():
    # Load the dataset.
    (x_train, y_train), (x_test, y_test) = datasets
    
    # Preprocess the images.
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Load the model.
    model = faster_rcnn(image_size=(224, 224), num_classes=4, num_landmarks=4)

    optimizer = Adam(learning_rate=0.0001)
    model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    print(len(x_train))
    print(len(y_train))
    print(len(x_test))
    print(len(y_test))

    model.fit( x_train, y_train, validation_data=(x_test, y_test),
    epochs=40,)

    model.save("model.h5")
    model.load_weights('model.h5')

    # Load the image.
    image = cv2.imread('./datasets/Rice_Leaf_Disease_Images_Dataset-4/RiceBacterialBlight/BB (50).jpg')

    # Predict the bounding box coordinates and landmarks.
    predictions = model.predict(image[np.newaxis, ...])
    bounding_box = predictions[0][0]
    #landmarks = predictions[0][2]

    # Draw the bounding box and landmarks on the image.
    image = draw_bounding_box(image, bounding_box)
    #image = draw_landmarks(image, landmarks)

    # Display the image.
    cv2.imshow('Image', image)
    cv2.waitKey(0)


if __name__ == '__main__':
    main()


151
151
32
32
Epoch 1/40


ValueError: in user code:

    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1332, in train_function  *
        return step_function(self, iterator)
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1316, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1297, in run_step  **
        outputs = model.train_step(data)
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1073, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1131, in compute_loss
        return self.compiled_loss(
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Step 1: Build the Backbone CNN (ResNet50 as an example)
def build_backbone(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
    intermediate_layer = base_model.get_layer('conv4_block6_out').output
    return intermediate_layer

# Step 2: Anchor Generation and RPN
def generate_anchor_boxes(base_size, aspect_ratios, scales):
    anchor_boxes = []
    for aspect_ratio in aspect_ratios:
        for scale in scales:
            width = base_size * np.sqrt(scale * aspect_ratio)
            height = base_size * np.sqrt(scale / aspect_ratio)
            anchor_boxes.append((width, height))
    return np.array(anchor_boxes)

def generate_anchors(feature_map_size, anchor_scale, aspect_ratios):
    base_anchor_size = anchor_scale  # You can adjust this value
    anchors = []
    for y in range(feature_map_size[0]):
        for x in range(feature_map_size[1]):
            center_x = (x + 0.5) / feature_map_size[1]
            center_y = (y + 0.5) / feature_map_size[0]
            for aspect_ratio in aspect_ratios:
                width = base_anchor_size * np.sqrt(aspect_ratio)
                height = base_anchor_size / np.sqrt(aspect_ratio)
                anchors.append([center_x, center_y, width, height])
    return np.array(anchors)

def build_rpn(input_layer, num_anchors):
    rpn_conv = Conv2D(256, (3, 3), activation='relu', padding='same')(input_layer)
    rpn_cls = Conv2D(num_anchors, (1, 1), activation='sigmoid', name='rpn_cls')(rpn_conv)
    rpn_reg = Conv2D(num_anchors * 4, (1, 1), activation='linear', name='rpn_reg')(rpn_conv)
    return rpn_cls, rpn_reg

# Step 3: ROI Pooling (or ROI Align)
class ROIPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, pooled_height, pooled_width, **kwargs):
        super(ROIPoolingLayer, self).__init__(**kwargs)
        self.pooled_height = pooled_height
        self.pooled_width = pooled_width

    def call(self, inputs):
        feature_maps, rois = inputs
        feature_maps = tf.cast(feature_maps, tf.float32)

        # Compute ROI dimensions and pooling bin sizes
        roi_height = rois[:, 3] - rois[:, 1]
        roi_width = rois[:, 2] - rois[:, 0]
        bin_size_h = roi_height / self.pooled_height
        bin_size_w = roi_width / self.pooled_width

        # Initialize pooled feature map
        pooled_features = []

        # Apply ROI Pooling for each ROI
        for i in range(tf.shape(rois)[0]):
            roi = rois[i]
            for ph in range(self.pooled_height):
                for pw in range(self.pooled_width):
                    h_start = tf.cast(tf.math.floor(roi[1] + ph * bin_size_h), dtype=tf.int32)
                    h_end = tf.cast(tf.math.ceil(roi[1] + (ph + 1) * bin_size_h), dtype=tf.int32)
                    w_start = tf.cast(tf.math.floor(roi[0] + pw * bin_size_w), dtype=tf.int32)
                    w_end = tf.cast(tf.math.ceil(roi[0] + (pw + 1) * bin_size_w), dtype=tf.int32)
                    pooled_region = feature_maps[h_start:h_end, w_start:w_end]
                    pooled_value = tf.reduce_max(pooled_region)
                    pooled_features.append(pooled_value)

        return tf.reshape(pooled_features, (-1, self.pooled_height, self.pooled_width, tf.shape(feature_maps)[-1]))
    

# Step 4: Fast R-CNN Head
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

def build_fast_rcnn(input_layer, num_classes):
    # Global Average Pooling
    pooled_features = GlobalAveragePooling2D()(input_layer)

    # Fully connected layers
    fc1 = Dense(256, activation='relu')(pooled_features)
    fc2 = Dense(256, activation='relu')(fc1)

    # Class prediction output
    cls_output = Dense(num_classes, activation='softmax', name='cls_output')(fc2)

    # Bounding box regression output
    num_bbox_regression_values = num_classes * 4
    reg_output = Dense(num_bbox_regression_values, activation='linear', name='reg_output')(fc2)

    return cls_output, reg_output

# Step 5: Loss Functions
def rpn_classification_loss(y_true, y_pred):
    rpn_cls_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return rpn_cls_loss

def rpn_regression_loss(y_true, y_pred, positive_indices):
    diff = y_true - y_pred
    absolute_diff = tf.abs(diff)
    mask = tf.cast(tf.less(absolute_diff, 1.0), tf.float32)
    rpn_reg_loss = tf.reduce_sum(mask * (0.5 * diff**2) + (1 - mask) * (absolute_diff - 0.5))
    return rpn_reg_loss

def fast_rcnn_classification_loss(y_true, y_pred):
    cls_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return cls_loss

def fast_rcnn_regression_loss(y_true, y_pred, positive_indices):
    diff = y_true - y_pred
    absolute_diff = tf.abs(diff)
    mask = tf.cast(tf.less(absolute_diff, 1.0), tf.float32)
    reg_loss = tf.reduce_sum(mask * (0.5 * diff**2) + (1 - mask) * (absolute_diff - 0.5))
    return reg_loss

# Step 6: Training Loop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

def train_step(images, rpn_cls_true, rpn_reg_true, cls_true, reg_true, model, optimizer):
    with tf.GradientTape() as tape:
        rpn_cls_pred, rpn_reg_pred, cls_pred, reg_pred = model(images, training=True)

        rpn_cls_loss = rpn_classification_loss(rpn_cls_true, rpn_cls_pred)
        rpn_reg_loss = rpn_regression_loss(rpn_reg_true, rpn_reg_pred, positive_indices_rpn)
        cls_loss = fast_rcnn_classification_loss(cls_true, cls_pred)
        reg_loss = fast_rcnn_regression_loss(reg_true, reg_pred, positive_indices_rcnn)

        total_loss = rpn_cls_loss + rpn_reg_loss + cls_loss + reg_loss

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss

def train(dataset, model, optimizer, epochs):
    for epoch in range(epochs):
        for batch_images, batch_rpn_cls_true, batch_rpn_reg_true, batch_cls_true, batch_reg_true in dataset:
            loss = train_step(batch_images, batch_rpn_cls_true, batch_rpn_reg_true, batch_cls_true, batch_reg_true, model, optimizer)
            print(f'Epoch {epoch + 1}, Batch Loss: {loss.numpy():.4f}')
        # Optionally, evaluate the model on validation set here


def main():
    # ... Load and preprocess your dataset ...

    input_shape = (224, 224, 3)
    num_classes = 4
    num_anchors = 9
    num_epochs = 10

    # Build the Faster R-CNN model
    input_layer = tf.keras.layers.Input(shape=input_shape)
    backbone_output = build_backbone(input_shape)
    rpn_cls, rpn_reg = build_rpn(backbone_output, num_anchors)


    pooled_height = 7  # Pooled feature map height
    pooled_width = 7   # Pooled feature map width
    #feature_maps_shape = (None, None, None, 256)  # Example feature map shape
    #rois_shape = (None, 4)  # Example ROIs shape

    # Create example inputs (feature maps and ROIs)
    #feature_maps = tf.random.normal(feature_maps_shape)
    rois = tf.constant([[20, 30, 140, 180], [100, 120, 220, 240]])

    feature_maps_shape = (2, 14, 14, 1024)  # Example feature map shape
    rois_shape = (None, 4)  # Example ROIs shape

    # Create example inputs (feature maps and ROIs)
    feature_maps = tf.random.normal(feature_maps_shape, dtype=tf.float32)  # Convert to float32
    rois = tf.constant([[20, 30, 140, 180], [100, 120, 220, 240]], dtype=tf.int32)  # Ensure int32


    # Build ROI Pooling layer
    #roi_pooling_layer = ROIPoolingLayer(pooled_height, pooled_width)
    roi_pooled_features = ROIPoolingLayer(pooled_height, pooled_width)([backbone_output, rois])  # Implement ROIPoolingLayer
    cls_output, reg_output = build_fast_rcnn(roi_pooled_features, num_classes)
    model = Model(inputs=input_layer, outputs=[rpn_cls, rpn_reg, cls_output, reg_output])

    # Compile the model and optimizer
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=[rpn_classification_loss, rpn_regression_loss,
                                             fast_rcnn_classification_loss, fast_rcnn_regression_loss])

    # Load and preprocess your dataset
    # train_dataset = DataLoader.load_train_dataset()  # Implement your data loading utilities
    (x_train,y_train),(_,_) = datasets
    train_dataset = (x_train,y_train)
    # Start training
    train(train_dataset, model, optimizer, num_epochs)

if __name__ == '__main__':
    main()


TypeError: Exception encountered when calling layer "roi_pooling_layer_2" (type ROIPoolingLayer).

in user code:

    File "C:\Users\HP 840 G1\AppData\Local\Temp\ipykernel_9376\3285425138.py", line 68, in call  *
        h_start = tf.cast(tf.math.floor(roi[1] + ph * bin_size_h), dtype=tf.int32)

    TypeError: Input 'y' of 'AddV2' Op has type float64 that does not match type int32 of argument 'x'.


Call arguments received by layer "roi_pooling_layer_2" (type ROIPoolingLayer):
  • inputs=['tf.Tensor(shape=(None, 14, 14, 1024), dtype=float32)', 'tf.Tensor(shape=(2, 4), dtype=int32)']

In [16]:
import cv2
import numpy as np

# Load segmented mask and original image
segmented_mask = cv2.imread('C:/Users/HP 840 G1/Documents/VS Code Projects/Workspace Learning/DataScience/Jupyter Notebook/Thesis_CSE\Masked.png', cv2.IMREAD_GRAYSCALE)
original_image = cv2.imread('C:/Users/HP 840 G1/Documents/VS Code Projects/Workspace Learning/DataScience/Jupyter Notebook/Thesis_CSE/original1.png')

# Find contours in the mask
contours, _ = cv2.findContours(segmented_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes on the original image
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the result
cv2.imshow('Image with Bounding Boxes', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
